Input

In [44]:
file_path = "input.txt"  # Path to the input file

with open(file_path, "r") as f:
    data = f.read()

inps, ops = data.split('\n\n')
inps = {z.split(':')[0]: int(z.split(':')[1].strip()) for z in inps.split('\n')}

ops = [op.split(' ') for op in ops.split('\n')]

Part 1

In [6]:
import collections

def solve_gate(inps, ops):
    not_visited = set([i for i in range(len(ops))])
    while len(not_visited):
        for v in not_visited:
            elem = ops[v]
            if elem[0] in inps and elem[2] in inps:
                op = elem[1]
                if op == 'AND':
                    inps[elem[4]] = inps[elem[0]] and inps[elem[2]]
                elif op == 'XOR':
                    inps[elem[4]] = inps[elem[0]] ^ inps[elem[2]]
                else:
                    inps[elem[4]] = inps[elem[0]] or inps[elem[2]]
                not_visited.remove(v)
                break
    od = collections.OrderedDict(sorted(inps.items()))
    rep = ''
    for k in od:
        if k.startswith('z'): rep += str(inps[k])
    return int(rep[::-1], 2)

In [7]:
solve_gate(inps, ops)

57344080719736

alternative solution to part 1 using z3

In [45]:
from z3 import *

solver = Solver()

z3_vars = {}

for var, value in inps.items():
    z3_vars[var] = Bool(var) if value is None else BoolVal(value)

for op in ops:
    input1 = z3_vars[op[0]] if op[0] in z3_vars else Bool(op[0])
    input2 = z3_vars[op[2]] if op[2] in z3_vars else Bool(op[2])
    output = op[4]
    
    if output not in z3_vars:
        z3_vars[output] = Bool(output)
    
    if op[1] == 'AND':
        solver.add(z3_vars[output] == And(input1, input2))
    elif op[1] == 'OR':
        solver.add(z3_vars[output] == Or(input1, input2))
    elif op[1] == 'XOR':
        solver.add(z3_vars[output] == Xor(input1, input2))

if solver.check() == sat:
    model = solver.model()

    z_values = [model[z3_vars[f'z{i:02}']] for i in range(46)]
    x_values = [z3_vars[f'x{i:02}'] for i in range(45)]
    y_values = [z3_vars[f'y{i:02}'] for i in range(45)]

    print(x_values)
    print(y_values)
    x = ''.join(['1' if t else '0' for t in reversed(x_values)])
    y = ''.join(['1' if t else '0' for t in reversed(y_values)])

    exp_result = bin(int(x, 2) + int(y, 2))
    result = ''.join(['1' if z else '0' for z in reversed(z_values)])

    print("Final binary result:", result)
    print("Expected binary result:", exp_result[2:])
    print("Integer value:", int(result, 2))
    print("Expected value:", int(exp_result[2:], 2))
else:
    print("No solution found")

[True, True, True, True, True, False, False, False, False, False, True, False, False, False, True, True, False, False, False, False, True, True, True, True, False, True, True, True, True, False, True, False, False, True, False, True, False, True, True, False, False, False, False, True, True]
[True, False, False, True, True, True, False, False, True, True, False, True, False, True, True, False, True, True, True, True, True, False, False, True, False, True, True, False, True, False, False, False, True, False, True, True, True, True, False, False, True, True, False, True, True]
Final binary result: 1101000010011101110101100100000010111101011000
Expected binary result: 1100111010011101110101100100000010111101011000
Integer value: 57344080752472
Expected value: 56794324938584


In [46]:
curr_str = "1101000010011101110101100100000010111101011000"[::-1]
exp_str = "1100111010011101110101100100000010111101011000"[::-1]

for i in range(len(curr_str)):
    if curr_str[i] != exp_str[i]:
        print(i) 
        break

39


Part 2: [Not solved yet]

In [16]:
import graphviz

# Create a new directed graph
dot = graphviz.Digraph(format='png', engine='dot')

# Add operand nodes (inputs/outputs) as circles
for var in inps:
    dot.node(var, var, shape='circle')  # Circle shape for operands

# Add operations as nodes (operators in boxes)
for op in ops:
    input1, gate, input2, _, output = op

    # Create gate node in a box
    gate_label = f'{gate}'
    gate_name = f'{input1}_{input2}_{gate}'  # Unique gate name
    dot.node(gate_name, gate_label, shape='box')  # Box shape for operators

    # Connect inputs to gate
    dot.edge(input1, gate_name)
    dot.edge(input2, gate_name)

    # Connect gate to output
    dot.node(output, output, shape='circle')  # Output node as circle
    dot.edge(gate_name, output)

# Render the graph to a file
output_path = 'circuit_graph_sample'
dot.render(output_path)  # Saves the output to 'circuit_graph.png'

'circuit_graph_sample.png'

In [19]:
# Not needed. Just for understanding
import graphviz


def visualize_full_adder_no_cin_graph(n):
    # Create graph
    dot = graphviz.Digraph(format='png', engine='dot')
    
    # Inputs
    for i in range(n):
        dot.node(f'A{i}', f'A[{i}]', shape='circle')  # Input A
        dot.node(f'B{i}', f'B[{i}]', shape='circle')  # Input B

    for i in range(n):
        # Internal gate nodes
        xor1 = f'XOR1_{i}'
        xor2 = f'XOR2_{i}'
        and1 = f'AND1_{i}'
        and2 = f'AND2_{i}'
        or1 = f'OR1_{i}'

        # Outputs
        sum_label = f'S{i}'
        carry_label = f'Cout{i+1}' if i < n - 1 else f'Cout{n}'

        # XOR1: A ⊕ B
        dot.node(xor1, 'XOR', shape='box')  # XOR gate
        dot.edge(f'A{i}', xor1)
        dot.edge(f'B{i}', xor1)

        # XOR2: (A ⊕ B) ⊕ Cin (implicitly Cin=0 for i=0)
        dot.node(xor2, 'XOR', shape='box')

        if i == 0:  # First bit has no initial carry-in
            dot.edge(xor1, xor2)  # Direct XOR result for Sum
        else:
            dot.edge(xor1, xor2)
            dot.edge(f'Cout{i}', xor2)  # Carry from previous stage

        # AND1: A ⋅ B
        dot.node(and1, 'AND', shape='box')
        dot.edge(f'A{i}', and1)
        dot.edge(f'B{i}', and1)

        # AND2: Cin ⋅ (A ⊕ B)
        if i > 0:  # For subsequent bits
            dot.node(and2, 'AND', shape='box')
            dot.edge(f'Cout{i}', and2)
            dot.edge(xor1, and2)

        # OR1: Carry-out
        dot.node(or1, 'OR', shape='box')
        dot.edge(and1, or1)
        if i > 0:  # For subsequent bits
            dot.edge(and2, or1)

        # Outputs
        dot.node(sum_label, sum_label, shape='circle')  # Sum output
        dot.edge(xor2, sum_label)

        dot.node(carry_label, carry_label, shape='circle')  # Carry output
        dot.edge(or1, carry_label)

    # Render and save the graph
    output_path = f'{n}-bit-adder-no-cin-graph'
    dot.render(output_path)


# Example: Visualize a 2-bit full adder with internals
visualize_full_adder_no_cin_graph(44)

In [43]:
res = ['svm', 'nbc', 'z39', 'fnr', 'z15', 'kqk', 'cgq', 'z23']
','.join(sorted(res))

'cgq,fnr,kqk,nbc,svm,z15,z23,z39'